In [101]:
import numpy as np
import pandas as pd
import xgboost
import yaml
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import shap
from sklearn.model_selection import StratifiedKFold

class generate_model():
    
    def __init__(self, predictor_col: str, data_path: str,xg_params: dict, kfold_splits : int , test_size=0.3, removed_features: list[str] = None, seed: float = None) -> None:
        if removed_features is None:
            removed_features = []
        dataset = pd.read_csv(data_path)

        # Variable encoding
        dataset["Género"] = dataset["Género"].replace({"M": 0, "F": 1}).astype("category")
        dataset["ATPII/AHA/IDF"] = (
            dataset["ATPII/AHA/IDF"].replace({"no": 0, "si": 1}).astype("category")
        )
        dataset["aleator"] = (
            dataset["aleator"]
            .replace({"Control": 0, "PKU 1": 1, "PKU 2": 2})
            .astype("category")
        )

        y_df = dataset[predictor_col].replace({"No": 0, "Si": 1}).astype("category")
        X_df = dataset.drop(predictor_col, axis="columns")

        X_df = X_df.drop(removed_features, axis="columns")

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                                                                X_df, y_df, stratify=y_df, test_size=test_size, random_state=seed
                                                            )
    #def xg_train(self, xg_params: dict, kfold_splits=5, seed=None):

        cv = StratifiedKFold(n_splits=kfold_splits, shuffle=True, random_state=seed)
        folds = list(cv.split(self.X_train, self.y_train))

        for train_idx, val_idx in folds:
            # Sub-empaquetado del train-set en formato de XGBoost
            dtrain = xgboost.DMatrix(
                self.X_train.iloc[train_idx, :],
                label=self.y_train.iloc[train_idx],
                enable_categorical=True,
            )
            dval = xgboost.DMatrix(
                self.X_train.iloc[val_idx, :],
                label=self.y_train.iloc[val_idx],
                enable_categorical=True,
            )

            self.model = xgboost.train(
                dtrain=dtrain,
                params=xg_params,
                evals=[(dtrain, "train"), (dval, "val")],
                num_boost_round=1000,
                verbose_eval=False,
                early_stopping_rounds=10,
            )
    def get_AUC_on_test_data(self):
        #def xg_test(model, X_test, y_test) -> float:
        testset = xgboost.DMatrix(self.X_test, label=self.y_test, enable_categorical=True)
        y_preds = self.model.predict(testset)

        return roc_auc_score(testset.get_label(), y_preds)
    
    
    def get_feature_metrics(self):
        # internal_feature_metrics = pd.DataFrame(
        #     {
        #         "Weight": self.model.get_score(importance_type="weight"),
        #         "Coverage": self.model.get_score(importance_type="cover"),
        #         "Gain": self.model.get_score(importance_type="gain"),
        #     }
        # ).sort_values(by="Gain", ascending=False)

        explainer = shap.TreeExplainer(self.model)

        # Extrae la explicacion SHAP en un DF
        EXPLAINATION = explainer(self.X_test).cohorts(
            self.y_test.replace({0: "Healty", 1: "Abnormal"}).tolist()
        )
        cohort_exps = list(EXPLAINATION.cohorts.values())

        exp_shap_abnormal = pd.DataFrame(
            cohort_exps[0].values, columns=cohort_exps[0].feature_names
        )  # .abs().mean()

        exp_shap_healty = pd.DataFrame(
            cohort_exps[1].values, columns=cohort_exps[1].feature_names
        )  # .abs().mean()


        feature_metrics = pd.concat(
            {
                "SHAP_healty": exp_shap_healty.abs().mean(),
                "SHAP_abnormal": exp_shap_abnormal.abs().mean(),
            },
            axis="columns",
        )

        # feature_metrics = (
        #     feature_metrics.join(internal_feature_metrics)
        #     .fillna(0)
        #     .sort_values(by="Gain", ascending=False)
        # )

        return feature_metrics
    
      
    
with open("params.yml", "r") as f:
        ext_params = yaml.load(f, Loader=yaml.FullLoader)
   
   
def objective(trial) -> float:
    """
    The function that runs a single model and evaluates it.
    """

    seed = trial.suggest_int("seed", 1, 10_000)

    # params={"objective":   "binary:logistic",
    #         "eval_metric": "logloss",
    #         'max_depth':   trial.suggest_int("max_depth", 2, 6, ),
    #         "eta":         trial.suggest_float("eta", 0.01, 0.3),
    #         "subsample":   trial.suggest_float("subsample", 0.5, 0.9),
    #         "lambda": trial.suggest_float("lambda", 0, 1),
    #         "alpha": trial.suggest_float("alpha",0,1),
    #         "scale_pos_weight": trial.suggest_float("scale_pos_weight",0,2)
    #     }
    
    params = {
            "objective": "binary:logistic",
            "eval_metric":       "logloss",
            "max_depth":                 5,
            "eta":                0.226908,
            "subsample":          0.749761,
            "lambda":             0.418621,
            "alpha":              0.466643,
            "scale_pos_weight":   1.640296,
    }
                
    model_instance = generate_model("HOMA-IR alterado", "data.csv", removed_features = ext_params["feature_engineering"]["removed_features"], xg_params = params, kfold_splits = 5, seed=seed)   
    
    return model_instance.get_AUC_on_test_data(), model_instance.get_feature_metrics()["SHAP_abnormal"]["fenilalax"]



In [102]:
import optuna
study_name: str = "alejandro_4"

study = optuna.create_study(
    study_name=study_name,
    storage="sqlite:///homa_studies.db",
    directions=["maximize", "maximize"],
    sampler=optuna.samplers.TPESampler(), # NSGAIISampler(),
    load_if_exists=True,
)

study.optimize(objective, n_trials=500, n_jobs=-1)
# Sampling may take a long time. Other samples are faster but tend to be hiperfixed and thus generate repeated results. 


[I 2023-05-03 09:08:08,410] A new study created in RDB with name: alejandro_4
[I 2023-05-03 09:08:09,791] Trial 4 finished with values: [0.6388888888888888, 0.18887652456760406] and parameters: {'seed': 9958}. 
[I 2023-05-03 09:08:09,923] Trial 5 finished with values: [0.8055555555555556, 0.0] and parameters: {'seed': 518}. 
[I 2023-05-03 09:08:10,308] Trial 0 finished with values: [0.7307692307692307, 0.0] and parameters: {'seed': 4405}. 
[I 2023-05-03 09:08:10,308] Trial 3 finished with values: [1.0, 0.0] and parameters: {'seed': 2242}. 
[I 2023-05-03 09:08:10,625] Trial 2 finished with values: [0.9615384615384616, 0.0] and parameters: {'seed': 2014}. 
[I 2023-05-03 09:08:10,626] Trial 7 finished with values: [0.38888888888888884, 0.0] and parameters: {'seed': 5688}. 
[I 2023-05-03 09:08:10,954] Trial 6 finished with values: [0.5384615384615384, 0.0] and parameters: {'seed': 1188}. 
[I 2023-05-03 09:08:10,954] Trial 1 finished with values: [0.5, 0.0] and parameters: {'seed': 7193}. 


In [103]:
import pandas as pd
from optuna.visualization._pareto_front import _get_pareto_front_info, _make_scatter_object, _make_marker, _make_hovertext
from typing import Sequence
from optuna.trial import FrozenTrial
from optuna.visualization._plotly_imports import go


info = _get_pareto_front_info(study)


n_targets: int = info.n_targets  
axis_order: Sequence[int]  = info.axis_order 
include_dominated_trials: bool = True
trials_with_values: Sequence[tuple[FrozenTrial, Sequence[float]]] = info.non_best_trials_with_values
hovertemplate: str = "%{text}<extra>Trial</extra>"
infeasible: bool = False
dominated_trials: bool = False

def trials_df(trials_with_values, class_name):
    x  =[values[axis_order[0]] for _, values in trials_with_values]
    y  =[values[axis_order[1]] for _, values in trials_with_values]

    df = pd.DataFrame({'x':x, 'y':y})
    df['class'] = class_name
    return df

df_best    = trials_df(info.best_trials_with_values, 'best')
df_nonbest = trials_df(info.non_best_trials_with_values, 'nonbest')
#df         = pd.concat([df_best, df_nonbest])

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_nonbest ['x'], y=df_nonbest['y'], mode='markers', marker=dict(size=8,opacity=0.5, symbol='circle', line=dict(width=1, color='black')), name='Suboptimal'))
fig.add_trace(go.Scatter(x=df_best['x'], y=df_best['y'], mode='markers', marker=dict(size=8,opacity=0.5, symbol='square', line=dict(width=1, color='black')), name='Pareto frontier'))
# Define the width and height of the plot
plot_width = 800
plot_height = 400

# Customize the plot
fig.update_layout(
    #title='Two-objective optimization',
    
    xaxis_title='AUC',
    yaxis_title='Phe importance',
    font=dict(size=14),
    #plot_bgcolor='white',
    # xaxis=dict(range=[min(df['x']) - 1, max(df['x']) + 1]),
    # yaxis=dict(range=[min(df['y']) - 1, max(df['y']) + 1]),
    legend=dict(x=0.3, y=1.1, bgcolor='rgba(255, 255, 255, 0)', orientation='h'),
    width=plot_width,
    height=plot_height,
    margin=dict(l=0, r=0, t=0, b=0)
)

# Show the plot
fig.show()

In [104]:
# # Extract the best trials from the multi-objective study
# best_trials = study.best_trials
# 
# # Print information about the best trials
# for i, trial in enumerate(best_trials):
#     print(f"Best trial {i+1} number: {trial.number}")
#     print(f"Best trial {i+1} values: {trial.values}")
#     print(f"Best trial {i+1} parameters: {trial.params}")
#     print()


# Ranking plots

In [105]:
study.best_trials

[FrozenTrial(number=96, state=TrialState.COMPLETE, values=[0.6944444444444444, 0.6909835934638977], datetime_start=datetime.datetime(2023, 5, 3, 9, 8, 21, 733904), datetime_complete=datetime.datetime(2023, 5, 3, 9, 8, 24, 139569), params={'seed': 1201}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'seed': IntDistribution(high=10000, log=False, low=1, step=1)}, trial_id=64848, value=None),
 FrozenTrial(number=221, state=TrialState.COMPLETE, values=[1.0, 0.3688807189464569], datetime_start=datetime.datetime(2023, 5, 3, 9, 8, 42, 19550), datetime_complete=datetime.datetime(2023, 5, 3, 9, 8, 44, 204382), params={'seed': 4528}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'seed': IntDistribution(high=10000, log=False, low=1, step=1)}, trial_id=64973, value=None),
 FrozenTrial(number=449, state=TrialState.COMPLETE, values=[0.7777777777777778, 0.6426613926887512], datetime_start=datetime.datetime(2023, 5, 3, 9, 9, 22, 854496), datetime_compl

In [109]:
study.get_trials()

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.7307692307692307, 0.0], datetime_start=datetime.datetime(2023, 5, 3, 9, 8, 8, 423223), datetime_complete=datetime.datetime(2023, 5, 3, 9, 8, 10, 42598), params={'seed': 4405}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'seed': IntDistribution(high=10000, log=False, low=1, step=1)}, trial_id=64752, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.5, 0.0], datetime_start=datetime.datetime(2023, 5, 3, 9, 8, 8, 429027), datetime_complete=datetime.datetime(2023, 5, 3, 9, 8, 10, 778155), params={'seed': 7193}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'seed': IntDistribution(high=10000, log=False, low=1, step=1)}, trial_id=64753, value=None),
 FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.9615384615384616, 0.0], datetime_start=datetime.datetime(2023, 5, 3, 9, 8, 8, 470747), datetime_complete=datetime.datetime(2023, 5, 3, 9, 8, 10, 411024), 

In [122]:
# Create a empty data frame for 
bs_trials = pd.DataFrame(columns=[
 'alpha',
 'eta',
 'lambda',
 'max_depth',
 'scale_pos_weight',
 'seed',
 'subsample',
 'auc', 'phe'
])

hardcoded_params = {
            "objective": "binary:logistic",
            "eval_metric":       "logloss",
            "max_depth":                 5,
            "eta":                0.226908,
            "subsample":          0.749761,
            "lambda":             0.418621,
            "alpha":              0.466643,
            "scale_pos_weight":   1.640296,
    }

CUTOFF = 0.00
CUTOFF_PHE = 0.20

for t in study.get_trials(): #[:499]:
    if t.values[0] > CUTOFF and t.values[1] > CUTOFF_PHE:
        #bs_trials.loc[t.number] = list(t.params.values()) + list(t.values)
        bs_trials.loc[t.number] = [0.466643, 0.226908, 0.418621, 5, 1.640296] + list(t.params.values()) + [0.749761] + list(t.values)

# for t in study.best_trials:
# for t in better_trials:
#     bs_trials.loc[t.number] = list(t.params.values()) + list(t.values)
#     # and saves them to rows in a dataframe

bs_trials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 10 to 468
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   alpha             28 non-null     float64
 1   eta               28 non-null     float64
 2   lambda            28 non-null     float64
 3   max_depth         28 non-null     float64
 4   scale_pos_weight  28 non-null     float64
 5   seed              28 non-null     float64
 6   subsample         28 non-null     float64
 7   auc               28 non-null     float64
 8   phe               28 non-null     float64
dtypes: float64(9)
memory usage: 2.2 KB


In [123]:
bs_trials = bs_trials.drop_duplicates()
bs_trials.describe().loc['50%']

alpha                  0.466643
eta                    0.226908
lambda                 0.418621
max_depth              5.000000
scale_pos_weight       1.640296
seed                4528.000000
subsample              0.749761
auc                    0.777778
phe                    0.286172
Name: 50%, dtype: float64

In [124]:
import plotly.express as px
#fig = px.parallel_coordinates(bs_trials)

fig = px.parallel_coordinates(
      bs_trials, 
      color="auc",
      dimensions=[
            'seed',
            'alpha',
            'eta',
            'lambda',
            'max_depth',
            'scale_pos_weight',
            'subsample',
            "phe", 
            "auc"
      ],)
fig.show()

In [125]:
import plotly.express as px

fig_boxplot = px.box(
    bs_trials.drop("seed", axis="columns"), 
    points="all", 
    facet_col="variable", 
)

fig_boxplot.update_xaxes(matches=None)
fig_boxplot.update_yaxes(matches=None)
fig_boxplot.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig_boxplot.show()

In [126]:
bs_trials['seed'], bs_trials['max_depth'] = bs_trials['seed'].astype('int'), bs_trials['max_depth'].astype('int')
bs_trials.head()


,alpha,eta,lambda,max_depth,scale_pos_weight,seed,subsample,auc,phe
10,0.466643,0.226908,0.418621,5,1.640296,8246,0.749761,0.611111,0.292153
17,0.466643,0.226908,0.418621,5,1.640296,9413,0.749761,0.833333,0.231811
24,0.466643,0.226908,0.418621,5,1.640296,7826,0.749761,0.923077,0.338496
39,0.466643,0.226908,0.418621,5,1.640296,3886,0.749761,0.307692,0.200398
70,0.466643,0.226908,0.418621,5,1.640296,1778,0.749761,0.923077,0.358984


In [127]:
params ={"objective":   "binary:logistic",
            "eval_metric": "logloss"}


params.update(
    bs_trials.drop(["auc","phe"], axis="columns").iloc[1].to_dict()
)

params

{'objective': 'binary:logistic',
 'eval_metric': 'logloss',
 'alpha': 0.466643,
 'eta': 0.226908,
 'lambda': 0.418621,
 'max_depth': 5.0,
 'scale_pos_weight': 1.640296,
 'seed': 9413.0,
 'subsample': 0.749761}

In [128]:
bs_trials

,alpha,eta,lambda,max_depth,scale_pos_weight,seed,subsample,auc,phe
10,0.466643,0.226908,0.418621,5,1.640296,8246,0.749761,0.611111,0.292153
17,0.466643,0.226908,0.418621,5,1.640296,9413,0.749761,0.833333,0.231811
24,0.466643,0.226908,0.418621,5,1.640296,7826,0.749761,0.923077,0.338496
39,0.466643,0.226908,0.418621,5,1.640296,3886,0.749761,0.307692,0.200398
70,0.466643,0.226908,0.418621,5,1.640296,1778,0.749761,0.923077,0.358984
71,0.466643,0.226908,0.418621,5,1.640296,7500,0.749761,0.846154,0.272889
80,0.466643,0.226908,0.418621,5,1.640296,1566,0.749761,0.692308,0.338371
96,0.466643,0.226908,0.418621,5,1.640296,1201,0.749761,0.694444,0.690984
127,0.466643,0.226908,0.418621,5,1.640296,1591,0.749761,0.750000,0.207866
129,0.466643,0.226908,0.418621,5,1.640296,1340,0.749761,0.694444,0.286172


In [129]:

paretos = {}

df_ranks = pd.DataFrame()

for number in bs_trials.index: 


    instance_params = bs_trials.loc[number][['max_depth','eta','subsample','lambda','alpha','scale_pos_weight']].to_dict()
    instance_params['max_depth'] = int(instance_params['max_depth'])

    instance_params['objective'] = 'binary:logistic'
    instance_params['eval_metric'] = 'logloss'

    print(instance_params)

    model_instance = generate_model(
        "HOMA-IR alterado", "data.csv", 
        removed_features = ext_params["feature_engineering"]["removed_features"], 
        xg_params = instance_params,
        kfold_splits = 5, 
        seed = int(bs_trials.loc[number]['seed'])
    )   
# 
    print(
        model_instance.model.attributes()
    )
    
    df_ranks[number] = model_instance.get_feature_metrics()["SHAP_abnormal"]

    print(
        'Phe_value : ', model_instance.get_feature_metrics()["SHAP_abnormal"]["fenilalax"], "\t",
        'Phe_ranking : ',model_instance.get_feature_metrics()["SHAP_abnormal"].rank(ascending=False)["fenilalax"]
    )

    print('\n')





{'max_depth': 5, 'eta': 0.226908, 'subsample': 0.749761, 'lambda': 0.418621, 'alpha': 0.466643, 'scale_pos_weight': 1.640296, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
{'best_iteration': '14', 'best_ntree_limit': '15', 'best_score': '0.27761588618159294'}
Phe_value :  0.29215348 	 Phe_ranking :  3.0


{'max_depth': 5, 'eta': 0.226908, 'subsample': 0.749761, 'lambda': 0.418621, 'alpha': 0.466643, 'scale_pos_weight': 1.640296, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
{'best_iteration': '34', 'best_ntree_limit': '35', 'best_score': '0.27833866017560166'}
Phe_value :  0.23181069 	 Phe_ranking :  9.0


{'max_depth': 5, 'eta': 0.226908, 'subsample': 0.749761, 'lambda': 0.418621, 'alpha': 0.466643, 'scale_pos_weight': 1.640296, 'objective': 'binary:logistic', 'eval_metric': 'logloss'}
{'best_iteration': '9', 'best_ntree_limit': '10', 'best_score': '0.3725244787832101'}
Phe_value :  0.338496 	 Phe_ranking :  5.0


{'max_depth': 5, 'eta': 0.226908, 'subsample': 

In [131]:
df_ranks_long = df_ranks.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index

# Create a grouped barplot using Plotly
fig = px.bar(df_ranks, barmode='group', template='ggplot2')
fig.update_xaxes(categoryorder='array', categoryarray=order)

# Show the plot
fig.show()

# fig = px.bar(, x='variable', y='value')
# fig.show()